In [1]:
import numpy as np
np.random.seed(1000)

import cv2
import os
from PIL import Image 
import keras

In [29]:
IMAGE_SIZE = 64

image_directory =  'E:/datasets/AtopicDemaDataset/'
extensions = ['jpg','png','jpeg']
dataset = []
label = []

In [31]:
infected_images = os.listdir(image_directory+ 'Infected/')
for i, image_name in enumerate(infected_images):
    real_extension = image_name.split('.')[1]
    if real_extension in extensions:
        image = cv2.imread(image_directory+ 'Infected/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((IMAGE_SIZE,IMAGE_SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [33]:
Uninfected_images = os.listdir(image_directory+ 'Uninfected/')
for i, image_name in enumerate(Uninfected_images):
    real_extension = image_name.split('.')[1]
    if real_extension in extensions:
        image = cv2.imread(image_directory+ 'Uninfected/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((IMAGE_SIZE,IMAGE_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [4]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [30]:
INPUT_SHAPE = (IMAGE_SIZE,IMAGE_SIZE,3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

In [10]:
conv1 = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(inp)

pool1 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)

conv2 = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(drop1)

pool2 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)#flatten the matrix to get it ready for dense layer

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)

hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)


In [11]:
model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248

In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train,X_test,y_train,y_test = train_test_split(dataset, to_categorical(np.array(label)), 
                                                test_size = 0.20, random_state = 0)  

In [39]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#to prevent overfitting
es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience= 3, verbose = 1, mode = 'auto')

mcp = ModelCheckpoint(monitor="val_accuracy", filepath="E:\datasets/malaria_model2.h5",verbose= 1, save_best_only=True , mode = 'auto')

x = [es,mcp]

In [41]:
f = model.fit(
    np.array(X_train),
    y_train,
    batch_size = 64,
    verbose = 1,
    epochs=25,
    validation_split=0.1,
    shuffle=False,
    callbacks=x
)

Epoch 1/25
36/36 [==============================] - ETA: 0s - loss: 0.0316 - accuracy: 0.9925
Epoch 1: val_accuracy did not improve from 0.90040
36/36 [==============================] - 17s 446ms/step - loss: 0.0316 - accuracy: 0.9925 - val_loss: 0.4262 - val_accuracy: 0.8884
Epoch 2/25
36/36 [==============================] - ETA: 0s - loss: 0.0215 - accuracy: 0.9938
Epoch 2: val_accuracy did not improve from 0.90040
36/36 [==============================] - 16s 416ms/step - loss: 0.0215 - accuracy: 0.9938 - val_loss: 0.4603 - val_accuracy: 0.8765
Epoch 3/25
36/36 [==============================] - ETA: 0s - loss: 0.0136 - accuracy: 0.9969
Epoch 3: val_accuracy did not improve from 0.90040
36/36 [==============================] - 14s 399ms/step - loss: 0.0136 - accuracy: 0.9969 - val_loss: 0.4765 - val_accuracy: 0.8765
Epoch 4/25
36/36 [==============================] - ETA: 0s - loss: 0.0123 - accuracy: 0.9978
Epoch 4: val_accuracy did not improve from 0.90040
36/36 [=================

In [43]:
print("Test Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(y_test))[1]*100))

20/20 [==============================] - 2s 119ms/step - loss: 0.3162 - accuracy: 0.9281
Test Accuracy: 92.81%
